In [1]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
ws = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ShubhamM3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ShubhamM3\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv("SMSSpamCollection", sep="\t", names=['Label',"Msg"])
df.head()

,Label,Msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
Label    5572 non-null object
Msg      5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [4]:
corpus = []
for i in range (0,len(df)):
    review = df["Msg"][i]
    review = re.sub('^a-zA-Z0-9'," ",review)
    review = review.lower()
    review = review.split()
    review = [ws.lemmatize(x) for x in review if x not in (stopwords.words('english'))]
    review = " ".join(review)
    corpus.append(review)    

In [5]:
len(corpus)

5572

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [42]:
cv = CountVectorizer(binary = True)
x = cv.fit_transform(corpus).toarray()
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [43]:
y = pd.get_dummies(df['Label'])
y.iloc[:,1].values

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [45]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=10)

In [46]:
rf = RandomForestClassifier().fit(x_train,y_train)
rf

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [47]:
y_pred = rf.predict(x_test)
acc_score = accuracy_score(y_test,y_pred)
acc_score

0.9605168700646087

In [48]:
cr = classification_report(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1215
           1       1.00      0.69      0.82       178

   micro avg       0.97      0.96      0.96      1393
   macro avg       0.98      0.85      0.90      1393
weighted avg       0.97      0.96      0.96      1393
 samples avg       0.96      0.96      0.96      1393



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
